In [1]:
import pandas as pd
import os
import csv
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
from scipy.interpolate import UnivariateSpline
from scipy.optimize import curve_fit
from scipy import interpolate, optimize

In [9]:
from math import atan2,degrees

In [3]:
from Experimentos import Experimentos

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc59C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc63C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc24C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc24C - flir.csv'


In [5]:
exp = Experimentos[30]

In [14]:
def labelLine(line,x,label=None,align=True,**kwargs):

    ax = line.axes
    xdata = line.get_xdata()
    ydata = line.get_ydata()

    if (x < xdata[0]) or (x > xdata[-1]):
        print('x label location is outside data range!')
        return

    #Find corresponding y co-ordinate and angle of the
    ip = 1
    for i in range(len(xdata)):
        if x < xdata[i]:
            ip = i
            break

    y = ydata[ip-1] + (ydata[ip]-ydata[ip-1])*(x-xdata[ip-1])/(xdata[ip]-xdata[ip-1])

    if not label:
        label = line.get_label()

    if align:
        #Compute the slope
        dx = xdata[ip] - xdata[ip-1]
        dy = ydata[ip] - ydata[ip-1]
        ang = degrees(atan2(dy,dx))

        #Transform to screen co-ordinates
        pt = np.array([x,y]).reshape((1,2))
        trans_angle = ax.transData.transform_angles(np.array((ang,)),pt)[0]

    else:
        trans_angle = 0

    #Set a bunch of keyword arguments
    if 'color' not in kwargs:
        kwargs['color'] = line.get_color()

    if ('horizontalalignment' not in kwargs) and ('ha' not in kwargs):
        kwargs['ha'] = 'center'

    if ('verticalalignment' not in kwargs) and ('va' not in kwargs):
        kwargs['va'] = 'center'

    if 'backgroundcolor' not in kwargs:
        kwargs['backgroundcolor'] = ax.get_facecolor()

    if 'clip_on' not in kwargs:
        kwargs['clip_on'] = True

    if 'zorder' not in kwargs:
        kwargs['zorder'] = 2.5

    ax.text(x,y,label,rotation=trans_angle,**kwargs)

In [18]:
def labelLines(lines,align=True,xvals=None,**kwargs):

    ax = lines[0].axes
    labLines = []
    labels = []

    #Take only the lines which have labels other than the default ones
    for line in lines:
        label = line.get_label()
        if "_line" not in label:
            labLines.append(line)
            labels.append(label)

    if xvals is None:
        xmin,xmax = ax.get_xlim()
        xvals = np.linspace(xmin,xmax,len(labLines)+2)[1:-1]

    for line,x,label in zip(labLines,xvals,labels):
        labelLine(line,x,label,align,**kwargs)

In [7]:
exp.eliminate_outliers().shape

(1948, 13)

In [6]:
exp.flir.shape

(1999, 11)

In [6]:
exp.flir.head()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9
0,0,1666,0.587635,0.063625,0.055222,0.120648,0.049820,0.070828,0.027611,0.024610,0.000000
1,1,2254,0.478261,0.053239,0.035492,0.066105,0.022626,0.068323,0.208518,0.067436,0.000000
2,2,3051,0.443461,0.043592,0.025565,0.033432,0.018355,0.033759,0.106850,0.234677,0.060308
3,3,3677,0.403590,0.039162,0.019581,0.014414,0.010606,0.027740,0.071798,0.221648,0.191460
4,4,3923,0.305379,0.037216,0.013765,0.011726,0.011471,0.023197,0.071119,0.253887,0.272241


In [21]:
fig = plt.plot(exp.flir['FLIR: tiempo'], exp.flir['FLIR: area'])

In [20]:
labelLines(plt.gca().get_lines(),align=False,color='k')

In [71]:
df = exp.flir.copy()

In [38]:
exp.flir.shape

(1999, 12)

In [60]:
df['moving average'] = df['FLIR: area'].rolling(20, center=True).median().fillna(method='bfill').fillna(method='ffill')

In [61]:
df['moving std'] = df['FLIR: area'].rolling(20, center=True).std().fillna(method='bfill').fillna(method='ffill')

In [67]:
df[np.abs(df['FLIR: area'] - df['moving average'])<=(2*df['moving std'])].shape

(1948, 13)

In [68]:
df = df[np.abs(df['FLIR: area'] - df['moving average'])<=(2*df['moving std'])]

In [70]:
fig = plt.plot(df['FLIR: tiempo'], df['FLIR: area'])